# Example of how to use cira with scikit-learn 

understand that this is just an example and not a good model for actual trading.

In this example we will setup cira and use Liner Regression as an model and try to make prediction on a crypto asset. 

### Cira setup

In [2]:
import cira

In [3]:
cira.auth.KEY_FILE = "../alpc_key.json"
assert cira.auth.check_keys(), "the set keys dose not work"

In [5]:
#portfolio = cira.Portfolio()
exchange = cira.Exchange()

### Fetching data 

In [6]:
from datetime import datetime

SYMBOL = "SPY"

cry = cira.Stock(SYMBOL)    

start = datetime(2020, 7, 1)
end = datetime(2023, 7, 1)

data = cry.historical_data_df(start_date=start, end_date=end)
data.head()

open    high     low   close      volume  \
symbol timestamp                                                               
SPY    2020-07-01 04:00:00+00:00  309.54  311.89  309.07  310.57  72474858.0   
       2020-07-02 04:00:00+00:00  314.24  315.70  311.51  312.19  69344217.0   
       2020-07-06 04:00:00+00:00  316.37  317.68  315.56  317.01  61713828.0   
       2020-07-07 04:00:00+00:00  315.38  317.52  313.37  313.77  82909978.0   
       2020-07-08 04:00:00+00:00  314.61  316.30  312.70  316.16  54638669.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2020-07-01 04:00:00+00:00     418495.0  310.486119  
       2020-07-02 04:00:00+00:00     433527.0  313.315661  
       2020-07-06 04:00:00+00:00     331248.0  316.169274  
       2020-07-07 04:00:00+00:00     398769.0  315.437016  
       2020-07-08 04:00:00+00:00     325899.0  314.892715

### Machine learning

training the model 

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


y = data[['close']]
X = data[['open', 'high', 'low']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 3.6098473671662688


using the models predictions

In [23]:
def trade_stocks(predictions, threshold=0.0):
    """ Buy if 1 sell -1 """
    BUY = 1 
    SELL = -1
    HOLD = 0
    positions = [BUY if pred > threshold else SELL for pred in predictions]
    positions = [HOLD if positions[i] == positions[i-1] else positions[i] for i in range(len(positions)) ]
    for i, position in enumerate(positions): 
        if position == BUY:
            break 
        if position == SELL:
            positions[i] = HOLD
    return positions

In [24]:
positions = trade_stocks(y_pred, threshold=400)
print(positions)

[0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 1, 0, 0, -1, 1, 0, -1, 0, 1, -1, 0, 0, 0, 0, 1, 0, -1, 0, 1, 0, -1, 0, 1, -1, 0, 0, 1, 0, 0, 0, -1, 0, 1, -1, 0, 0, 1, 0, -1, 0, 0, 1, 0, -1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 1, -1, 1, -1, 0, 1, 0, -1, 1, 0]
